In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls

gdrive	sample_data


In [ ]:
!ls gdrive

MyDrive  Shareddrives


In [ ]:
import os 
os.chdir("/content/gdrive/My Drive/imza_tanima/sign_samples")
!pwd

/content/gdrive/My Drive/imza_tanima/sign_samples


In [ ]:
import numpy as np
import pandas as pd
import glob
import random
import itertools

In [ ]:
icdar_dir="/content/gdrive/My Drive/imza_tanima/sign_samples/sign_data/train/"

In [ ]:
def create_kaggle_data(dir,n_class):
  files=glob.glob(dir+'/*')
  org_files=[]
  forg_files=[]
  
  org_data = []
  forg_data = []
  org_labels = []
  forg_labels = []
  for i in range(len(files)):
    if files[i].rsplit('_')[-1] =='forg':
      #print(files[i])
      forg_files.append(files[i])
    else:
      org_files.append(files[i])

  for i in range(n_class):
    org_data += glob.glob(org_files[i]+'/*')
    forg_data += glob.glob(forg_files[i]+'/*')

  forg_img_names=[file.rsplit('/')[-2]+'/'+file.rsplit('/')[-1] for file in forg_data]
  org_img_names=[file.rsplit('/')[-2]+'/'+file.rsplit('/')[-1] for file in org_data]


  for i in range(len(org_img_names)):
    class_index = int(org_img_names[i].split('/',1)[0])
    org_labels.append(class_index - 1)
  for i in range(len(forg_img_names)):
    class_index = int(forg_img_names[i].split('/',1)[0].split('_')[0])
    forg_labels.append(class_index - 1)
    
  return np.array(org_img_names), np.array(org_labels),np.array(forg_img_names), np.array(forg_labels)#, nb_classes

In [ ]:
def pairs(g_path,f_path,org_indices,forg_indices,classes,per):
  train_pairs = []
  train_labels = []
  test_pairs = []
  test_labels = []
  
  for d in range(classes): 

    if (d <= int(classes*per)):
      a1=org_indices[d]
      a2=random.sample(list(org_indices[d]),5)
      b=random.sample(list(forg_indices[d]),5)

      test_f=list(itertools.product(a1,b))
      test_g=list(itertools.product(a1,a2))

      for i in range(len(test_f)):
        c1, c2 = test_f[i][0], test_f[i][1]
        test_pairs += [[g_path[c1],f_path[c2]]]
        test_labels += [1]
        
      for i in range(len(test_g)):
        d1, d2 = test_g[i][0], test_g[i][1]
        if d1 != d2:
          test_pairs += [[g_path[d1],g_path[d2]]]
          test_labels += [0]

      test_data=pd.DataFrame(test_pairs)
      test_data.columns = ['image1','image2']
      test_data['labels']= test_labels

    else:
      a1=org_indices[d]
      a2=random.sample(list(org_indices[d]),5)
      b=random.sample(list(forg_indices[d]),5)

      train_f=list(itertools.product(a1,b))
      train_g=list(itertools.product(a1,a2))

      for i in range(len(train_f)):
        a1, a2 = train_f[i][0], train_f[i][1]
        train_pairs += [[g_path[a1],f_path[a2]]]
        train_labels += [1]

      for i in range(len(train_g)):
        b1, b2 = train_g[i][0], train_g[i][1]
        if b1 != b2:
          train_pairs += [[g_path[b1],g_path[b2]]]
          train_labels += [0]
  
      train_data=pd.DataFrame(train_pairs)
      train_data.columns = ['image1','image2']
      train_data['labels']= train_labels
      
  return train_data, test_data

In [ ]:
def pairs_to_csv(dir,n_class,per):
  if dir == cedar_path :
    forg_dir = cedar_path + "/full_forg"
    org_dir = cedar_path + "/full_org"

    org_path, org_y= create_data(org_dir)
    forg_path, forg_y= create_data(forg_dir)
  elif dir==icdar_dir:
    org_path, org_y, forg_path, forg_y = create_kaggle_data(dir,n_class)
  else:
    org_path, org_y, forg_path, forg_y = create_bh_data(dir,n_class)

  org_indices = [np.where(org_y == i)[0] for i in range(n_class)]
  forg_indices = [np.where(forg_y == i)[0] for i in range(n_class)]

  train, test = pairs(org_path, forg_path, org_indices, forg_indices, n_class, per)
  
   train.to_csv(path_or_buf = dir + 'train_pairs.csv', index=False)
   test.to_csv(path_or_buf = dir + 'test_pairs.csv', index=False)

  print(f'Train samples: {train.shape[0]} \nTest samples: {test.shape[0]}')

In [ ]:
pairs_to_csv(icdar_dir,64,0.2)

Train samples: 5865 
Test samples: 2685
